 ###                 MULTIVARIATE LINEAR REGRESSION

This notebook will show how employee salaries can be predicted from different employee characteristics (or features).

In [6]:
#Required Packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats
from sklearn.linear_model import LinearRegression 
import statsmodels
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings("ignore")

#### **This assignment addresses the following:**


[Exploratory Data Analysis -EDA](#EDA) 
<a href='EDA'> </a>

[Feature Engineering](#feng)
<a href='feng'> </a>

[Correlation and Statistical Significance Analysis](#corr)
<a href='corr'> </a>

[Feature Selection](#select)
<a href='select'> </a>

[Model Training](#train)
<a href='train'> </a>

[Predictions](#predict)
<a href='predict'> </a>


[Model Evaluation](#eval)
<a href='eval'> </a>


<a id='EDA'></a> <div class="alert alert-block alert-info">

#### Exploratory Data Analysis (EDA)
- Data Ingestion
- Data Preprocessing
 </div>

In [7]:
#Load Data
sal = pd.read_csv('data/salary.csv')

In [ ]:
#Examine DataSet
sal.info()

* Our target variable (salary) has one missing value.

In [ ]:
#Method to fix missing value
sal['salary'].fillna(sal['salary'].mean(),inplace = True)
#sal = sal.drop_duplicates()

* Missing value has been filled w/ mean. Duplicates would be dropped if dataset had any 
* Filled w/ mean instead of dropping because salary is the target variable and don't want to miss any insights it might bring even when it's just one value.

In [ ]:
#convert floats to ints
sal = sal.astype(int)

Now that the missing value has been dealt with I have converted all the columns to integers (info method above shows us that 2 columns (salary&market) are floats.) I chose to convert because down the line when we calculate metrics and compare our predictions, it's better to work with whole numbers versus floats that need to be rounded off.

__Note that astype() will raise an error when it encounters NaN values so be sure to deal w/ them first before proceeding__

In [ ]:
sal.describe()

* The above table tells us the number of observations (514). We expect salary to have 514 because the missing value is now represented by the mean.
* The descriptive statistics table also tells us about the means, standard deviations, min and max values as well as the percentiles.


 <a id='feng'></a> <div class="alert alert-block alert-info">

#### Feature Engineering
 </div>

In [ ]:
import plotly.graph_objs as go

field = ['Engineering', 'Finance', 'Human Resources', 'Marketing']
trace = go.Pie(labels = field, values = sal.Field)
data = [trace]
layout = go.Layout(
   {"title":"Career Fields"})
fig = go.Figure(data,layout)
fig.show()

The purpose of this graph is to have an understanding of which career field are the most dominant within the dataset. The Marketing profession accounts for 33.3%, Eng & HR are tied at second place leaving Finance in last.

In [ ]:
dummies = pd.get_dummies(sal['Field'])

dum = dummies.rename(columns={1:'engineering',2:'finance',3:'HR',4:'marketing'})

sal2 = pd.concat([sal, dum] ,axis=1, ignore_index=False)
sal2.drop("Field", inplace=True, axis=1)

 <a id='corr'></a> <div class="alert alert-block alert-info">

#### Correlation Analysis and Statistical Significance 
 </div>

### Calculate Pearson correlation coeffificent and plot the corresponding correlation matrix


In [ ]:
#omitting categorical variables
sal_corr = sal.drop(["Field", 'position'], axis=1)

"For a dichotomous categorical variable and a continuous variable you can calculate a Pearson correlation if the categorical variable has a 0/1-coding for the categories. ... But when you have more than two categories for the categorical variable the Pearson correlation is not appropriate anymore."
ref: https://www.researchgate.net/post/Can_I_use_Pearsons_correlation_coefficient_to_know_the_relation_between_perception_and_gender_age_income#:~:text=For%20a%20dichotomous%20categorical%20variable,1%2Dcoding%20for%20the%20categories.&text=But%20when%20you%20have%20more,correlation%20is%20not%20appropriate%20anymore.

In [ ]:
sal_corr

In [ ]:
sal_corr.corr()
sal_corr.style.background_gradient(cmap = 'Greys')

__Everything has a correlation but it is the strength of the correlation that we are interested in.__
* Based on the above table these features are good predictors for salary; yearsworked, yearsrank, position and Field


In [ ]:
print(f'p-value & correlation coef of yearsworked and salary: {stats.pearsonr(sal.yearsworked, sal.salary)}')
print(f'p-value & correlation coef of yearsrank and salary: {stats.pearsonr(sal.yearsrank, sal.salary)}')
print(f'p-value & correlation coef of postion and salary: {stats.pearsonr(sal.position, sal.salary)}')
print(f'p-value & correlation coef of Field and salary: {stats.pearsonr(sal.Field, sal.salary)}')

<a id='select'></a> <div class="alert alert-block alert-info">

#### Feature Selection
 </div>

In [ ]:
rank={1:'Junior', 2:'Manager', 3:'Executive'}
sal2['position']=sal2.position.map(rank)

In [ ]:
sal2 = sal.drop(['exprior', 'market', 'degree', 'otherqual', 'male', 'yearsabs'], axis=1)

## How To Check for Multicollinearity

<blockquote> "Multicollinearity exists whenever an independent variable is highly correlated with one or more of the other independent variables in a multiple regression equation." - (Taken from the book 'Understanding Regression Analysis')
</blockquote>

- Step 1: Check correlation between selected features. (i.e identify variables affected by multicollinearity)
- Step 2: Calculate the VIF factors
(The Variance Inflation Factor (VIF) is a measure of colinearity among predictor variables within a multiple regression.)
- Step 3: Inspect the factors for each predictor variable, if the VIF is between 5-10, multicolinearity is likely present and you should consider dropping the variable.


#### [Read More Here](https://newprairiepress.org/cgi/viewcontent.cgi?referer=https://www.google.com/&httpsredir=1&article=1034&context=agstatconference)

In [ ]:
#Step1
m_coll = sal2[['yearsworked', 'yearsrank', 'position', 'Field']]

In [ ]:
#Step2

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

#calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(m_coll.values, i) for i in range(m_coll.shape[1])]
vif["features"] = m_coll.columns

In [ ]:
vif.round(1)

In [ ]:
#Step 3

###### Interpretaion

- VIF ranges from 1 upwards. 
- generally a VIF above 10 indicates high correlation 
- if you have high VIFs for dummy variables representing nominal variables with three or more categories, those are usually not a problem.

Judging by the correlation matrix below, yearsworked is highly correlated with position (0.75) & yearsrank (0.81). This is the variable that'll be removed before moving on to model building.

In [ ]:
features_corr = sal2.corr()
features_corr.style.background_gradient(cmap = 'Oranges')

In [ ]:
sal2 = sal2.drop(['yearsworked'], axis=1)

 <a id='train'></a> <div class="alert alert-block alert-info">

#### Model Training
 </div>

__Dataset is split into a train & test set. 
70% of the data will go into the training set and the remaining 30% will be used for testing.__

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
#split data
x = sal2.drop('salary', axis=1)
y = sal2["salary"]

x = sm.add_constant(x)

X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.3, random_state=50)

In [ ]:
model = ols('salary ~ yearsrank + position + Field', sal2).fit()

In [ ]:
model.summary()

R-squared value reveals the quality of the regression model. It describes the relationships between dependent and independent variables in a model. The R-squared value for this model is 0.66. This means the accuracy of the model is approx 66%.

<a id='predict'></a> <div class="alert alert-block alert-info">

#### Predictions & Model Testing

Predicting salary using the test set.
 </div>

In [ ]:
predict_salary = model.predict(X_test)

We use the __X_test__ data to pass in features the model has never seen

In [ ]:
#dataframe for predicted values
df = pd.DataFrame(predict_salary)
df.rename(columns={0:'predicted_salary'},inplace=True)
df = df.astype(int)
df.head(2)

In [ ]:
df_compare = pd.concat([Y_test,df],axis=1)

In [ ]:
df_compare

- The model has made reasonable predictions. Lets look at the figures in index 214 for example. Salary had a value of 59 472 and the model predicted 59 570! That's Incredible!
- The model has an accuracy rate of 66% and some estimations may be off. The values in index 206 are one such example.
- A conclusion we can make from this is that although the model needs some fine-tuning, the chosen predictors can help in predicting a person's salary.
- The model can be generalized.

### Analyse Distribution of Residuals

In [ ]:
sns.distplot((Y_test - predict_salary))
plt.title("Distribution of Residuals")
plt.show()

- The figure is normally distributed
- This is a good sign because it means this model is a correct choice for the data



<a id='eval'></a> <div class="alert alert-block alert-info">

#### Model Evaluations
Regression Evaluation Metrics

 </div>

#### 3 common evaluation metrics for regression problems:

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

**Mean Squared Error** (MSE) is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

**Root Mean Squared Error** (RMSE) is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

All of these are **loss functions**. We want to minimize them to create the best model.

In [ ]:
from sklearn import metrics

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print('MSE:', int(metrics.mean_squared_error(Y_test, predict_salary)))

In [ ]:
print('testRMSE:', int(np.sqrt(metrics.mean_squared_error(Y_test, predict_salary))))

In [ ]:
print('trainsetRMSE:', int(np.sqrt(metrics.mean_squared_error(Y_train, model.predict(X_train)))))

- RMSE indicates the absolute fit of the model to the data (how close the actual data points are to the model's predicted ones.)
- Test RMSE is bigger than the Trainset RMSE
- RMSE is a good measure of how accurately the model predicts the target variable. It is the most important criteria for fit if the main purpose of the model is prediction.

#### Mean Absolute Error(MAE) is one of the many metrics for summarizing and assessing the quality of a machine learning model.


In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE:', int(metrics.mean_absolute_error(Y_test, predict_salary)))

In [ ]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mape(Y_test, predict_salary)

- The mean absolute percentage error (MAPE) is a statistical measure of how accurate a forecast system is. It measures this accuracy as a percentage.
- (MAPE) works best if there are no extremes to the data
- Since MAPE is a measure of error, high numbers are bad and low numbers are good (meaning that our 12.2% is something to be happy about)


In [ ]:
metrics.r2_score(Y_test, predict_salary)

### Prediction Error Plot

In [ ]:
from sklearn.linear_model import Lasso
from yellowbrick.regressor import PredictionError

In [ ]:
mod = Lasso()
visualizer = PredictionError(mod)

#Fit training data to visualizer
visualizer.fit(X_train, Y_train)

#Evaluate model on the test data
visualizer.score(X_test, Y_test)

visualizer.show()                 

- A __Prediction Error Plot__ shows the actual vales from the dataset against the predicted values generated by the model.
- This allows us to see how much variance is in the model.
- The Line of Best Fit is used to express a relationship in a scatter plot of different data points.
- It is an output of regression analysis and can be used as a prediction tool for indicators.

In [ ]:
import statsmodels.api as sm

### Calculate the standardised residuals resid() & standardised predicted values fittedvalues()

In [ ]:
residuals  = model.resid

fittedv   = model.fittedvalues

#### Plot the residuals versus the predicted values using seaborn’s residplot with fitted values as the x parameter, and the dependent variable as y, specify lowess=True. 


In [ ]:
sns.residplot(fittedv,residuals, lowess=True, color='maroon')
plt.title('Residuals vs Predicted Values')
plt.ylabel('Fitted Values')
plt.xlabel('Residuals')
plt.show()